In [ ]:
import matplotlib.pyplot as plt
plt.plot([1,2,3,4])

In [ ]:
import os
import numpy as np
import torch
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline

import torch.optim as optim

In [ ]:
root = './data'
trans = transforms.ToTensor()
train_set = dset.MNIST(root, train=True,transform=trans,download=False)

In [ ]:
train_set[0]

In [ ]:
plt.imshow(train_set[0][0].data.numpy()[0]);

In [ ]:
len(train_set)

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_loader = DataLoader(dataset=train_set,batch_size=64)

In [ ]:
test_set = dset.MNIST(root, train=False,transform=trans,download=True)
test_loader = DataLoader(test_set,batch_size=1)
len(test_set)

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        self.conv1 = nn.Conv2d(1,32,3)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32,65,3)
        self.conv3 = nn.Conv2d(65,64,3)
        self.fc1 = nn.Linear(576,64)
        self.fc2 = nn.Linear(64,10)
        
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = x.reshape(x.size(0),-1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

        
        
        
        

In [ ]:
conv = ConvNet()
print(conv)

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(conv.parameters(),lr=0.01,momentum=0.9)

In [ ]:
losses = []
for epoch in range(3):
    running_loss = 0.0
    
    for i, data in enumerate(train_loader):

        inputs, labels = data
    
        optimizer.zero_grad()
        
        outputs = conv(inputs)
        loss = loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i % 100 == 99:
            losses.append(running_loss)
            print('[%d,%d], loss: %.3f' % (epoch + 1, i+1,running_loss/100))
            running_loss = 0.0

In [ ]:
plt.plot(losses)

In [ ]:
results = []
for im, label in test_loader:

    prediction = np.argmax(conv(im).detach().numpy())
    if prediction == label:
        results.append(1)
    else:
        results.append(0)
        

In [ ]:
np.sum(results)/10000